## Comparing the MSE of Bayes vs Frequentist regression

In [1]:
import pandas as pd
import numpy as np
import pymc as pm

In [2]:
data = pd.read_csv('wr_model_data.csv')

In [3]:
data = data.dropna()

In [4]:
data['P_five'] = data['P_five'].astype(object)
data

Year  Round  Pick  Overall                 Name        Team Position  \
1    2010      1    24       24           Dez Bryant     COWBOYS       WR   
2    2010      2     7       39       Arrelious Benn  BUCCANEERS       WR   
3    2010      2    28       60          Golden Tate    SEAHAWKS       WR   
5    2010      3    14       78       Brandon LaFell    PANTHERS       WR   
6    2010      3    18       82     Emmanuel Sanders    STEELERS       WR   
..    ...    ...   ...      ...                  ...         ...      ...   
273  2022      3    24       88        Jalen Tolbert     COWBOYS       WR   
274  2022      3    35       99           David Bell      BROWNS       WR   
276  2022      4    20      125        Erik Ezukanma    DOLPHINS       WR   
279  2022      4    43      148        Khalil Shakir       BILLS       WR   
280  2022      5    19      162  Montrell Washington     BRONCOS       WR   

                 School  Rec.Yards  Rec.TD   Rec  CONF P_five Pos   RAS  \
1        Oklahoma State     1480.0    19.0  87.0   B12      1  WR  9.05   
2              Illinois     1055.0     3.0  67.0   B10      1  WR  8.23   
3            Notre Dame     1496.0    15.0  93.0   ACC      1  WR  7.49   
5       Louisiana State      929.0     8.0  63.0   SEC      1  WR  4.88   
6    Southern Methodist     1339.0     7.0  98.0  CUSA      0  WR  9.38   
..                  ...        ...     ...   ...   ...    ...  ..   ...   
273       South Alabama     1474.0     8.0  82.0    SB      0  WR  8.62   
274              Purdue     1286.0     6.0  93.0   B10      1  WR  3.99   
276          Texas Tech      748.0     6.0  46.0   B12      1  WR  7.03   
279         Boise State     1117.0     7.0  77.0   NP5      0  WR  8.31   
280             Samford      813.0     9.0  60.0    SC      0  WR  4.32   

     AllTime  
1       9.17  
2       8.21  
3       7.43  
5       4.94  
6       9.41  
..       ...  
273     8.62  
274     3.99  
276     7.03  
279     8.31  
280     4.32  

[216 rows x 16 columns]

In [11]:
data_train = data[data['Year'] < 2020]
data_test = data[data['Year'] >=2020]

In [12]:
X = data_train[['Rec.Yards','Rec.TD','Rec','RAS']]
Y = data_train['Overall']

meanx = X.mean().values
scalex = X.std().values
zX = ((X - meanx)/scalex).values

meany = Y.mean()
scaley = Y.std()
zY = ((Y - meany)/scaley).values

In [18]:
X_test = data_test[['Rec.Yards','Rec.TD','Rec','RAS']]
Y_test = data_test['Overall']

meanx_test = X_test.mean().values
scalex_test = X_test.std().values
zX_test = ((X_test - meanx_test)/scalex_test).values

meany_test = Y_test.mean()
scaley_test = Y_test.std()
zY_test = ((Y_test - meany_test)/scaley_test).values

In [15]:
with pm.Model() as wr_model:
    Y = pm.MutableData(name = 'Y', value = zY)
    X1 = pm.MutableData(name = 'X1', value = zX[:,0])
    X2 = pm.MutableData(name = 'X2', value = zX[:,1])
    X3 = pm.MutableData(name = 'X3', value = zX[:,2])
    X4 = pm.MutableData(name = 'X4', value = zX[:,3])
    
    beta0 = pm.Normal('beta0', mu=0, sigma=2) 
    beta1 = pm.Normal('beta1', mu=0, sigma=2) 
    beta2 = pm.Normal('beta2', mu=0, sigma=2)
    beta3 = pm.Normal('beta3', mu=0, sigma=2)
    beta4 = pm.Normal('beta4', mu=0, sigma=2)
    beta5 = pm.Normal('beta5', mu=0, sigma=2)
    mu = beta0 + beta1*X1 + beta2*X2 + beta3*X3 + beta4*X4
    
    nu = pm.Exponential('nu', 1/29.)
    sigma = pm.Uniform('sigma', 10**-5, 10)
    
    likelihood = pm.StudentT('likelihood', nu=nu, mu=mu, lam=1/sigma**2, observed = Y)
    trace = pm.sample(1000, cores = 4,target_accept=0.95)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1, beta2, beta3, beta4, beta5, nu, sigma]


/Users/connornickol/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/connornickol/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/connornickol/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/connornickol/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 74 seconds.


In [16]:
with wr_model:
    trace.extend(pm.sample_prior_predictive())
    trace.extend(pm.sample_posterior_predictive(trace))

Sampling: [beta0, beta1, beta2, beta3, beta4, beta5, likelihood, nu, sigma]
Sampling: [likelihood]


In [19]:
pm.set_data(new_data={'X1':zX_test[:,0],'X2':zX_test[:,1],'X3':zX_test[:,2],'X4':zX_test[:,3], 'Y':zY_test}, model=wr_model)
ppc_power = pm.sample_posterior_predictive(trace, model=wr_model, var_names=['likelihood'])

Sampling: [likelihood]


In [20]:
predict_p = pd.DataFrame({
    'predict':ppc_power['posterior_predictive']['likelihood'].mean(axis=1)[0].to_numpy()})

In [31]:
data_test = data_test.reset_index()
data_test['predict'] = predict_p

In [32]:
## MSE
((data_test['Overall'] - data_test['predict'])**2).mean()

9553.955723068026

In [33]:
data_test['predict'] = data_test['predict']

In [34]:
data_test['actual'] = zY_test

In [35]:
data_test

level_0  index  Year  Round  Pick  Overall                  Name  \
0         0    211  2020      1    15       15           Jerry Jeudy   
1         1    212  2020      1    17       17           CeeDee Lamb   
2         2    213  2020      1    21       21          Jalen Reagor   
3         3    214  2020      1    22       22      Justin Jefferson   
4         4    215  2020      1    25       25         Brandon Aiyuk   
5         5    216  2020      2     1       33           Tee Higgins   
6         6    220  2020      2    17       49        Chase Claypool   
7         7    222  2020      2    27       59           Denzel Mims   
8         8    223  2020      3     2       66        Antonio Gibson   
9         9    226  2020      3    28       92        Devin Duvernay   
10       10    227  2020      4    22      128         Gabriel Davis   
11       11    228  2020      4    36      142  Antonio Gandy-Golden   
12       12    229  2020      5     5      151              Joe Reed   
13       13    233  2020      5    22      168        John Hightower   
14       14    234  2020      5    25      171        Isaiah Coulter   
15       15    235  2020      5    27      173        Darnell Mooney   
16       16    240  2021      1    20       20        Kadarius Toney   
17       17    241  2021      1    27       27        Rashod Bateman   
18       18    242  2021      2     2       34          Elijah Moore   
19       19    243  2021      2    17       49         Rondale Moore   
20       20    245  2021      2    25       57           Tutu Atwell   
21       21    247  2021      3    13       77           Josh Palmer   
22       22    248  2021      3    18       82           Dyami Brown   
23       23    249  2021      3    21       85         Amari Rodgers   
24       24    250  2021      3    25       89          Nico Collins   
25       25    251  2021      3    27       91      Anthony Schwartz   
26       26    254  2021      4    24      129         Jaelon Darden   
27       27    255  2021      4    26      131         Tylan Wallace   
28       28    256  2021      5    13      157  Ihmir Smith-Marsette   
29       29    257  2021      5    35      179           Simi Fehoko   
30       30    258  2021      5    37      181        Cornell Powell   
31       31    260  2022      1    10       10        Garrett Wilson   
32       32    261  2022      1    11       11           Chris Olave   
33       33    263  2022      1    15       16          Jahan Dotson   
34       34    264  2022      1    18       18         Treylon Burks   
35       35    266  2022      2    11       43     Wan'Dale Robinson   
36       36    268  2022      2    18       50       Tyquan Thornton   
37       37    269  2022      2    20       52        George Pickens   
38       38    270  2022      2    21       53           Alec Pierce   
39       39    271  2022      2    22       54            Skyy Moore   
40       40    273  2022      3    24       88         Jalen Tolbert   
41       41    274  2022      3    35       99            David Bell   
42       42    276  2022      4    20      125         Erik Ezukanma   
43       43    279  2022      4    43      148         Khalil Shakir   
44       44    280  2022      5    19      162   Montrell Washington   

          Team Position            School  Rec.Yards  Rec.TD    Rec  CONF  \
0      BRONCOS       WR           Alabama     1315.0    14.0   68.0   SEC   
1      COWBOYS       WR          Oklahoma     1327.0    14.0   62.0   B12   
2       EAGLES       WR   Texas Christian     1061.0     9.0   72.0   B12   
3      VIKINGS       WR   Louisiana State     1540.0    18.0  111.0   SEC   
4        49ERS       WR     Arizona State     1192.0    16.0   65.0   P12   
5      BENGALS       WR           Clemson     1167.0    13.0   59.0   ACC   
6     STEELERS       WR        Notre Dame     1037.0    13.0   66.0   ACC   
7         JETS       WR            Baylor     1087.0     8.0   61.0   B12  